# Affinity Propagationモデルを使った株式市場のサブグループ検出

Affinity Propagation（アフィニティ・プロパゲーション）

クラスタ数を事前に指定する事なく、クラスタリングをしてくれる。

- 入力で使う教師なしデータ
    - exampler

- クラスタリングの評価値
    - responsibility
    - avilability

`pandas_datareader`をインストールしておかないといけない

```
$ pip install pandas_datareader
```

In [12]:
import datetime
import json
import numpy as np
from sklearn import covariance, cluster
import pandas as pd
import pandas_datareader.data as pdd

input_file = './data/company_symbol_mapping.json'
with open(input_file, 'r') as f:
    company_symbol_map = json.loads(f.read())

# JSONファイルのKeyだけを読み込む
symbols = company_symbol_map.keys()


## 株価データを取得する

- [Quandle](https://www.quandl.com/)というサイトから読み込むためにユーザ登録してAPI KEYを取得する必要がある。
